# Hakai POM QA-QC 
---
# Start Notebook
## Install packages 
Google colab servers has already a few commonly used packages installed. To install those missing, which are specific toh Hakai let's run the following commands:

In [2]:
!pip install hakai-api
!pip install git+git://github.com/HakaiInstitute/ioos_qc.git@development
!pip install git+git://github.com/HakaiInstitute/process_ocean_timeseries

  Cloning git://github.com/HakaiInstitute/ioos_qc.git (to revision development) to /tmp/pip-req-build-8xsd9lmn
  Running command git clone -q git://github.com/HakaiInstitute/ioos_qc.git /tmp/pip-req-build-8xsd9lmn
  Running command git checkout -b development --track origin/development
  Switched to a new branch 'development'
  Branch 'development' set up to track remote branch 'development' from 'origin'.
     |████████████████████████████████| 1.0MB 3.8MB/s 
     |████████████████████████████████| 112kB 8.3MB/s 
     |████████████████████████████████| 133kB 7.5MB/s 
     |████████████████████████████████| 6.6MB 8.2MB/s 
     |████████████████████████████████| 15.4MB 258kB/s 
     |████████████████████████████████| 552kB 29.5MB/s 
  Created wheel for ioos-qc: filename=ioos_qc-1.0.0-cp37-none-any.whl size=44537 sha256=afd63608fb06422e6466d870f6519dffaa113172b7b7e5339bfbdbffa143bbd7
  Stored in directory: /tmp/pip-ephem-wheel-cache-4pdf55_w/wheels/2f/bb/7b/19f2997af03515525ae518c26716a8

## Import the python packages needed

In [3]:
# Let's load pandas for working with the data in table
import pandas as pd 
import numpy as np

# Let's load seaborn to plot the data
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# This to install the hakai api tool and be able to download some data from hakai's database
from hakai_api import Client

# Install ioos_qc which is used to qc data
from ioos_qc.config import QcConfig
from ioos_qc import qartod

from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets, HBox, VBox
import time

from process_ocean_data.tools import qc as review

# Define some project standards

In [4]:
# And the interesting variables to use from the Hakai database
nutrient_variables = ['corr_delta15n','ug_c','corr_delta13c']

pd.options.display.max_columns = 100

# Download data from the Hakai database
For more information regarding the Hakai API, go [here](https://github.com/HakaiInstitute/hakai-api).

You can find a list of all the data type endpoints [here](http://hakaiinstitute.github.io/hakai-api/#endpoints).

In [5]:
# Let's retrieve the endpoint to retrieve nutrients data:
endpointUrl = '/eims/views/output/poms'
site_id = 'QU39'
start_time = '2015-01-01'
end_time = '2022-06-11'
# We'll retrieve data only associated with QU39 between January 1st 2019 to January 1st 2020
filterUrl = 'site_id={0}&collected>{1}&collected<{2}&limit=-1'.format(
    site_id, start_time, end_time
)

# Get Hakai Data    
#Get Data from Hakai API
client = Client() # Follow stdout prompts to get an API token

# Make a data request for sampling stations
url = '%s/%s?%s' % (client.api_root,endpointUrl,filterUrl)
response = client.get(url)
df = pd.DataFrame(response.json())
original_columns = df.columns
print(str(len(df))+' records downloaded')
df.head()

Please go here and authorize:
https://hecate.hakai.org/api-client-login


KeyboardInterrupt: ignored

In [ ]:
# Let's convert the collected time to a datetime object variable called  time 
#  and extract the from those datetime objects the year and month 
df['time'] = pd.to_datetime(df['collected'])
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['dayoftheyear'] = pd.to_timedelta(df['time'].dt.dayofyear, unit='d')

# Define a depth variable which is: 
#   - pressure_transducer_depth (if available)
#   - OR line_out_depth
df['depth'] = df['pressure_transducer_depth'].fillna(df['line_out_depth'])

# Review Replicates
## Pool Standard Deviation


In [ ]:
 # Let's create a pooled standard deviation function
def pooled_standard_deviation(df_to_review,count_col='count',std_col='std'):
    # Keep only records that have replicates
    df_replicates = df_to_review[df_to_review[count_col]>1]
    upper =  df_replicates[count_col].sub(-1).mul(df_replicates[std_col].pow(2)).sum()
    lower = df_replicates[count_col].sub(-1).sum()
    pooled_std = np.sqrt(upper/lower)
    return pooled_std

In [ ]:
df_grouped = df.groupby(['site_id','line_out_depth','collected']).agg(['mean','std','count'])

for var in nutrient_variables:
    pool_std = pooled_standard_deviation(df_grouped[var])
    print('{0} pool.std.: {1}'.format(var,pool_std))

corr_delta15n pool.std.: 1.5094291555267862
ug_c pool.std.: 51.48868056209912
corr_delta13c pool.std.: 0.6734276279567056


## Replicates Standard Deviation Distribution
Present the distribution of the standard deviations the replicate samples for each line_out_depth.

In [ ]:
px.histogram(df_grouped['ug_c'].reset_index().dropna(subset=['std']).sort_values('line_out_depth'),
             x='std',color='line_out_depth', hover_name='collected',
             marginal='box')

# Apply detection limit flag
Samples with values lower than the dectection limit will be flag as BDL.



In [ ]:
# # If lower than detection limit, flag as BDL
# df.loc[df['no2_no3_um']<0.036 ,'no2_no3_um_bdl_flag']='BDL'
# df.loc[df['po4']<0.032 ,'po4_bdl_flag']='BDL'
# df.loc[df['sio2']<0.1 ,'sio2_bdl_flag']='BDL'

# Time series QARTOD tests
Review each depth time series for a station and run timeseries specific test on them. The configuarion dictionary below, list of each variables and depth ranges the tests that will be applied.


## Set QARTOD Tests Configuration

In [ ]:
qc_config = [{
    "depth_range":{
        "minimum":-5,
        "maximum":55
        },
    "streams":{
        "no2_no3_um":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 36],
                    "fail_span": [0, 40]
                    },
                "aggregate": {}
            }
        },
        "po4":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 3],
                    "fail_span": [0, 4]
                    },
                "aggregate": {}
            }
                    
        },
        "sio2":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 80],
                    "fail_span": [0, 100]
                    },
                "aggregate": {}
            }
                    
        }
    }
},{
    "depth_range":{
        "minimum":55,
        "maximum":300
        },
    "streams":{
        "no2_no3_um":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 36],
                    "fail_span": [0, 40]
                    },
                "spike_test": {
                    "suspect_threshold": 2,
                    "fail_threshold": 3,
                    "method": "differential",
                    "n_dev_suspect":1,
                    "n_dev_fail":2,
                    "n_records":40,
                    "min_records":20
                    },
                "aggregate": {}
            }
        },
        "po4":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 3],
                    "fail_span": [0, 4]
                    },
                "spike_test": {
                    "suspect_threshold": 0.5,
                    "fail_threshold": 1,
                    "method": "differential",
                    "n_dev_suspect":2,
                    "n_dev_fail":3,
                    "n_records":50,
                    "min_records":40
                    },
                "aggregate": {}
            }
                    
        },
        "sio2":{
            "qartod": {
                "gross_range_test":{
                    "suspect_span": [0, 80],
                    "fail_span": [0, 100]
                    },
                "spike_test": {
                    "suspect_threshold":8,
                    "fail_threshold": 12,
                    "method": "differential",
                    "n_dev_suspect":2,
                    "n_dev_fail":3,
                    "n_records":30,
                    "min_records":20
                    },
                "aggregate": {}
            }
                    
        }
    }
}]


## Run QARTOD Tests

In [ ]:
# Run QARTOD tests
# We are using the deprecated QcConfig method and hopefully will move 
#  to a new stream method soon.
time = 'time'
depth = 'depth'
group_timeseries = ['line_out_depth']
for item in qc_config:
    df_depth_range = df[(df['depth']>item['depth_range']["minimum"]) & \
                        (df['depth']<item['depth_range']["maximum"])]
    for line_out_depth, timeserie in df_depth_range.groupby(['site_id','line_out_depth']):
        timeserie = timeserie.sort_values(time)
        for var in item['streams'].keys():
            qc = QcConfig(item['streams'][var])
            qc_result = qc.run(
                inp=timeserie[var],
                tinp=timeserie[time],
                zinp=timeserie[depth],
            )
            for module,tests in qc_result.items():
                for test, flag in tests.items():
                    df.loc[timeserie.index, var+'_'+module+"_"+test] = flag

# Map QARTOD Flags to Hakai Convention
for var in df.filter(like='qartod').columns:
     df[var] = df[var].replace({pd.NA:9}).astype(int).replace(review.flag_conventions['mapping']['QARTOD-HAKAI'])

# Review Interannual Variability
Let's compute the average value measured for each depth and the associated standard deviation.

## Compute the seasonal variability
### Monthly Values


In [ ]:
# Compute the monthly value recorded for each station and line_out_depth 
df_seasonal = df.groupby(['site_id','line_out_depth','month']) \
  .agg(['mean','std'])[nutrient_variables] \
  .reset_index()
df_seasonal.columns = ['_'.join(filter(None,col)).strip() for col in df_seasonal.columns.values]

# Add a seasonal value to each data
df_with_seasons = pd.merge(df,df_seasonal,on=['site_id','line_out_depth','month'], suffixes=('','_seasonal'))


### Over 60 days window

In [ ]:
# TODO not that pandas.rolling isn't compatible with center window yet.
# Or get the season with a 30 days running window instead
window = '60d'
df_running_seasonal = df.groupby(['site_id','line_out_depth']).apply(
    lambda x: x.sort_values('dayoftheyear').set_index('dayoftheyear')[nutrient_variables]\
       .rolling(window).agg(['mean','std']).reset_index()
)
df_running_seasonal.columns = ['_'.join(filter(None,col)).strip() for col in df_running_seasonal.columns.values]

# Since rolling is looking the days prior, we'll center the value to the middle of the window.
#   rolling(center) isn't yet available for datetime index
df_running_seasonal['dayoftheyear'] = df_running_seasonal['dayoftheyear']-pd.to_timedelta(window)/2
df_running_seasonal.loc[df_running_seasonal['dayoftheyear']<pd.to_timedelta('0d'),'dayoftheyear'] += pd.to_timedelta('365d')

# Clean the matrix a bit
df_running_seasonal = df_running_seasonal.reset_index()\
    .drop('level_2',axis='columns')\
    .set_index(['site_id','line_out_depth','dayoftheyear'])\
    .add_suffix('_seasonal')

# Add a seasonal value to each data
df = pd.merge(df,df_running_seasonal,
                           on=['site_id','line_out_depth','dayoftheyear'])


## Make Monthly Box Plots

In [ ]:
var = widgets.ToggleButtons(
    options=nutrient_variables,
    value=nutrient_variables[0],
    description="Select Variable",
    disabled=False,
)
def make_montly_box_plot(var):
    # Show the seasonality of the data
    fig = px.box(df.sort_values('line_out_depth'),x='month',y=var,
            color='line_out_depth', animation_frame='line_out_depth',
        hover_name='hakai_id',
        width=1200, height=600)
    return fig 
interact(make_montly_box_plot, var=var)

interactive(children=(ToggleButtons(description='Select Variable', options=('corr_delta15n', 'ug_c', 'corr_del…

<function __main__.make_montly_box_plot>

# Create Aggregated Suggested Flag


In [ ]:
flags_considered = ['_qartod_aggregate','_bdl_flag']
for var in nutrient_variables:
    flag_vars = [var+item for item in flags_considered]
    df[var+'_review_flag'] = \
       df[flag_vars].apply(lambda x: review.compare_flags(list(x),convention='HAKAI'),axis='columns')

# Update Original Flags
df['po4_flag'] = df['po4_flag'].fillna(df['po4_review_flag'])
df['sio2_flag'] = df['sio2_flag'].fillna(df['sio2_review_flag'])
df['no2_no3_flag'] = df['no2_no3_flag'].fillna(df['no2_no3_um_review_flag'])

KeyError: ignored

# Review Results Manually

In [ ]:
# Plot profile
group_profile_by = 'date'
profile_list = widgets.SelectMultiple(
    options=df[group_profile_by].sort_values().drop_duplicates(),
    value=(df[group_profile_by].iloc[0],),
    description="Select Profiles",
    disabled=False,
)
def make_profile_plot(profiles):
    # Filter only selected profiles
    df_selected = df[df['date'].isin(profiles)]
    df_temp = df[df['date'].isin(profile_list.value)].sort_values(['date','depth'])
    fig = []
    for ii in [0,1,2]:
        fig.append(px.line(df_temp,x=nutrient_variables[ii],y='depth',color='date',hover_data=['hakai_id']))

    subfig = make_subplots(rows=1, cols=3)
    kk=1
    for item in fig:
        for subitem in item.data:
            if kk>1:
                subitem.showlegend=False
            subfig.add_trace(subitem,row=1,col=kk)
        subfig.update_xaxes(title_text=nutrient_variables[kk-1], row=1, col=kk)
        kk += 1
    subfig.update_yaxes(title_text='Depth (m)', row=1, col=1)
    subfig.update_yaxes(autorange="reversed")
    subfig.update_xaxes(matches=None)
    subfig.update_traces(mode='markers+lines')
    return subfig

interact(make_profile_plot, profiles=profile_list)

interactive(children=(SelectMultiple(description='Select Profiles', index=(0,), options=('2015-03-18', '2015-0…

<function __main__.make_profile_plot>

In [ ]:
group_timeseries_by = 'line_out_depth'
# Plot TimeSeries
timeseries_id = widgets.SelectMultiple(
    options=df[group_timeseries_by].sort_values().drop_duplicates().values,
    value=(df[group_timeseries_by].iloc[0],),
    description="Select "+group_timeseries_by,
    disabled=False,
)

nutrient_id = widgets.ToggleButtons(
    options=nutrient_variables,
    value=nutrient_variables[0],
    description="Select "+group_timeseries_by,
    disabled=False,
)
timeseries_id
def plot_timeseries(nutrient, profile):
    flag_columns = df.filter(regex= nutrient+'.*(flag|qartod)').columns
    df_temp = df[df['line_out_depth'].isin(timeseries_id.value)].sort_values('time').copy()

    fig = px.scatter(df_temp,
            x='time',
            y=nutrient,
            hover_data=['date','hakai_id','depth']+list(flag_columns),
            color_discrete_map={value:item['Color'] for value, item in review.flag_conventions['HAKAI'].items()}
           )

    return fig
    
interact(plot_timeseries,nutrient=nutrient_id, profile=timeseries_id)

interactive(children=(ToggleButtons(description='Select line_out_depth', options=('corr_delta15n', 'ug_c', 'co…

<function __main__.plot_timeseries>

In [ ]:
# Review QC Manually (Not compatible with Colab)
review.manual_qc_interface(df, ['no2_no3','sio2','po4'], 'HAKAI' ,review_flag="_flag")


# Save Result to Hakai Portal Compatible Excel Format

In [ ]:
df_output = df[original_columns]

# Rename Columns to Match Portal Output
# Drop unchanged rows

###ATTENTION### The output has not yet been tested with the Hakai API.
df_output.to_excel('Hakai_Nutrient_Revision_{0}.xlsx'.format(pd.Timestamp.now().isoformat().replace(':','')))


# Report Figures

## Contour plot

In [ ]:

# Plot contourf interpolate linearly over the x axis and maximum over two NaN values
var = 'delt'
df_pivot = pd.pivot_table(df,values=var,index='line_out_depth',columns='date',aggfunc='mean').interpolate(axis='index',limit=4).sort_index(axis=0).sort_index(axis=1).interpolate(axis='columns',limit=3)
fig = go.Figure(data =
    go.Contour(z=df_pivot.values,x=df_pivot.columns,y=df_pivot.index.values,
               colorbar=dict(title=var, titleside='right'),
               colorscale='RdYlGn',
               ncontours=10,
               contours_coloring='heatmap'
               #,connectgaps=True
              ))
fig.update_yaxes(title='Depth (m)',autorange="reversed",linecolor='black',mirror=True,
                 ticks='outside',showline=True)
fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
fig.show()

## Scatter with colorbar

In [ ]:
## Scatter with colorbar
fig = px.scatter(df.dropna(subset=['no2_no3_um'],axis=0),x='time',y='depth',
                 color='no2_no3_um',color_continuous_scale='RdYlGn',
                 hover_name='hakai_id')
fig.update_yaxes(title='Depth (m)',autorange="reversed",linecolor='black',mirror=True,
                 ticks='outside')
fig.update_xaxes(linecolor='black',mirror=True,ticks='outside',showline=True)
fig.show()